In [46]:
import queue as q
import copy as cp
from queue import PriorityQueue
import numpy as np

In [47]:
#this is essentially the setting up of the board:{remember, the rules for where the numbers can be moved is not 
#given, it is at the descretion of the user to enter the numbers right and do the right actions}
# e -stands for empty
#goal state:
# 1 2 3
# 4 5 6
# 7 8 e 
class Puzzle8:

#some global variables which corresponds to the whole problem tree and not just the node

    #Number of nodes in the graph
    node_num = 1
    
    #maximum number of nodes in priority queue at any point (mostly for potential analytical purposes)
    max_fringe_len = 1
    
    #set of explored nodes, to avoid nodes
    explored = set()
    
    def __init__(self, parent):
        self.parent = parent

#checks if a given state is solvable, uses inversion check, specific to 8 puzzle problem.
    def is_solvable(self):
        a = []
        inv = 0
        for i in range(1,10):
            val = self.number_pos[i] 
            if(val == 'e'):
                a.append(0)
            else:
                a.append(val)
        
        for i in range(0,8):
            for j in range(i+1,9):
                if a[i] and a[j] and (a[i] > a[j]):
                    inv += 1
                    
        return ((inv%2) != 1)            
            
            
#defining equality for objects of the 8Puzzle class     
    def __eq__(self,other):
        a1 = (self.number_pos[1] == other.number_pos[1])
        a2 = (self.number_pos[2] == other.number_pos[2])
        a3 = (self.number_pos[3] == other.number_pos[3])
        a4 = (self.number_pos[4] == other.number_pos[4])
        a5 = (self.number_pos[5] == other.number_pos[5])
        a6 = (self.number_pos[6] == other.number_pos[6])
        a7 = (self.number_pos[7] == other.number_pos[7])
        a8 = (self.number_pos[8] == other.number_pos[8])
        a9 = (self.number_pos[9] == other.number_pos[9])
        return (a1 and a2 and a3 and a4 and a5 and a6 and a7 and a8 and a9)

#defining lesser than for the objects of the class, this function is necessary for adding it to heaps and queues.
    def __lt__(self, other):
        return False
    

#outputs all possible moves from the given state
#['l' - moves tile to the left, 'r' - moves tile to the right, 'u' - moves tile up, 'd' - moves tile down]
    def possible_moves(self):        
        for i in range(1,10):
          if(self.number_pos[i] == 'e'):
            # empty position in the puzzle
            emt_pos = i
            break

        if(emt_pos == 1):
            return ['l','u']
        if(emt_pos == 2):
            return ['r','l','u']
        if(emt_pos == 3):
            return ['r','u']
        if(emt_pos == 4):
            return ['l','u','d']
        if(emt_pos == 5):
            return ['r','l','u','d']
        if(emt_pos == 6):
            return ['r','u','d']
        if(emt_pos == 7):
            return ['l','d']
        if(emt_pos == 8):
            return ['r','l','d']
        if(emt_pos == 9):
            return ['r','d']
        
"""Note: the methods are named with empty slot movement in mind aka a tile moving left is akin to empty slot moving right"""
#moves a tile from right of empty slot to empty slot    
    def mov_left(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'e'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos+1]
        self.number_pos[emt_pos+1] = 'e'

#moves a tile from left of empty slot to empty slot
    def mov_right(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'e'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos-1]
        self.number_pos[emt_pos-1] = 'e'

#moves a tile from down(bottom) of empty slot to empty slot
    def mov_up(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'e'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos+3]
        self.number_pos[emt_pos+3] = 'e'

#moves a tile from up(top) of empty slot to empty slot
    def mov_down(self):
        for i in range(1,10):
          if(self.number_pos[i] == 'e'):
            emt_pos = i
        self.number_pos[emt_pos] = self.number_pos[emt_pos-3]
        self.number_pos[emt_pos-3] = 'e'

#prints the board represented by the object
    def show_board(self):
        
        print(self.number_pos[1],self.number_pos[2],self.number_pos[3])
        print(self.number_pos[4],self.number_pos[5],self.number_pos[6])
        print(self.number_pos[7],self.number_pos[8],self.number_pos[9])

#returns the board as a dictionary
    def get_board(self):
        res = self.number_pos.copy()
        return res

#checks if the state already exists in the explored set
    def exist(self):
        return self.dict_to_str() in Puzzle8.explored
    
#number of misplaced tiles(mismatch count)
    def heur1(self):
        h1 = -1
        for i in range(1,10):
            if(self.number_pos[i] != i):
                h1 += 1
        return h1

#number of squares a tile is away from its goal state(manhattan distance)
    def heur2(self):
        h2 = 0
        for i in range(1,10):
            #print(i)
            if(self.number_pos[i] != 'e'):
                # find the horizontal and vertical values of the tile if it was at its intended position
                # we are also going to make use of the fact that the dictionary keys also refer to the correct tile number
                # for its position, which is why we use 'i' ( which ranges from 1 to 9 including both ends, same as the keys of
                # our state dictionary - number_pos)
                k = i%3
                if(k == 0):
                    num_pos_h = 3
                    num_pos_v = int(i/3)
                else:
                    num_pos_h = k
                    num_pos_v = int(i/3) + 1
                
                # find the horizontal and vertical value of the tile for the current positions of the tile
                p = self.number_pos[i]%3
                if(p == 0):
                    a_num_pos_h = 3
                    a_num_pos_v = int(self.number_pos[i]/3)
                else:
                    a_num_pos_h = p
                    a_num_pos_v = int(self.number_pos[i]/3) + 1
                man_dist = abs(num_pos_h-a_num_pos_h) + abs(num_pos_v-a_num_pos_v)
                h2 += man_dist               
        return h2

#converts dictionary(board representation) into a string to push into a set(so that it can be hashed)
#returns the string
    def dict_to_str(self):
        result = ''
        for i in range(1,10):
            result = result+str(self.number_pos[i])
        return result

#checks if a state is in its goal state and return True if it is, False otherwise
    def goal_test(self):
        for i in range(1,9):
            if(self.number_pos[i] != i):
                return False
        return True

IndentationError: unexpected indent (3137913756.py, line 91)

In [48]:
#returns all the unexplored children of the node as a list of Puzzle8 objects
def expand(S):
    pos_moves = S.possible_moves()
    children_nodes = []
 
    #The possible moves are iterated through, and the object from the resulting moves are created.
    for i in range(pos_moves.__len__()):
        if pos_moves[i] == 'r':
            child = Puzzle8(S)
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.mov_right()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.add(child.dict_to_str())
                children_nodes.append(child)
        elif pos_moves[i] == 'l':
            child = Puzzle8(S)
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.mov_left()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.add(child.dict_to_str())
                children_nodes.append(child)
        elif pos_moves[i] == 'u':
            child = Puzzle8(S)
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.mov_up()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.add(child.dict_to_str())
                children_nodes.append(child)
        else:
            child = Puzzle8(S)
            child.number_pos = S.get_board()
            child.depth = S.depth + 1
            child.mov_down()
            if not(child.exist()):
                Puzzle8.node_num += 1
                Puzzle8.explored.add(child.dict_to_str())
                children_nodes.append(child)
    return children_nodes

#converts a list into a queue and returns the queue
def make_fifo_queue(lis):
    result = q.Queue()
    [result.put(ele) for ele in lis]
    return result


#modified BFS search which checks upto a given depth and returns a queue with all the elements.
def mod_exp(S,depth):
    temp3_1 = []
    child_list = expand(S)
    temp1 = make_fifo_queue(child_list)
    temp2 = make_fifo_queue(child_list)
    d = depth - 1
    while d > 0:
        while temp2.qsize() != 0:
            C = temp2.get()
            temp3_1 = expand(C)
            temp3_2 = make_fifo_queue(temp3_1)
            while temp3_2.qsize() !=0:
                temp1.put(temp3_2.get())
        d -= 1
        temp2 = make_fifo_queue(temp3_1)
    return temp1
        

#modified A* search, the "AI" part
def mod_A(initial_state,heur,depth=1):
    root = Puzzle8(None)
    root.number_pos = initial_state
    root.depth = 0
    root.parent = None
    
    Puzzle8.explored.add(root.dict_to_str())
    fringe = PriorityQueue()
    S = root
    print("initial state:")
    root.show_board()
    while not(S.goal_test()):
        # get all the nodes upto a certain depth
        temp = mod_exp(S,depth)
        
        # Iterate through the queue:
        while temp.qsize() != 0:
            
            # Pop first element from queue
            l = temp.get()
            """note: the val here is cost of moving to the specific node, the heur1 and heur2 are two different
               heuristic costs used for testing the code. 
               l.depth is equivalent to the path cost. Each depth adding 1"""
            
            # get the cost value for the node
            # two different heuristics we can set with heur flag
            if heur == 1:
                # mismatch count heuristic used
                val = (l.heur1()+l.depth)
            else:
                # manhattan distance heuristic used
                val = (l.heur2()+l.depth)
 
            # add them to the priority queue
            fringe.put((val,l))
                
        # store the max fringe size, only for analytical purpose
        Puzzle8.max_fringe_len = max(Puzzle8.max_fringe_len,fringe.qsize())
        
        # as long as the fringe size is not 0
        if(fringe.qsize() != 0):
            # pop the top element from the priority queue and continue iteration from that point
            # this essentially moves the graph search to the node with least cost, since priority makes sure of that.
            S = fringe.get()[1]
        else:
            # if there are no values in the fringe, break out of the loop, only happens if there are no solutions, 
            # after iterating, all possible moveset
            break
    #returns solution object(or last object checked if there is no solution)
    return S

#creates 10 random states in Puzzle8 acceptable format and return is as a list of dictionaries
#Also makes sure they are solvable
def create_states():
    i=0
    result = []
    base_arr = ['e',1,2,3,4,5,6,7,8]
    while i<10:
        np.random.shuffle(base_arr)
        state = {
            1:base_arr[0],
            2:base_arr[1],
            3:base_arr[2],
            4:base_arr[3],
            5:base_arr[4],
            6:base_arr[5],
            7:base_arr[6],
            8:base_arr[7],
            9:base_arr[8],    
        }
        p = Puzzle8(None)
        p.number_pos = state
        if(p.is_solvable()):
            result.append(state)
            i += 1
        del p
            
    return result

def reproducePath(S):
    temp = cp.deepcopy(S)
    while temp.__class__.__name__ == "Puzzle8":
        temp.show_board()
        print("_________")
        temp = temp.parent

In [49]:
#driver function going to be called
def driver():
    
    state = create_states()
    
    print("solving 10 states with depth = 1, 'mismatch count' heuristic:")
    print(" ")
    for i in range(10):
        Puzzle8.node_num = 1
        Puzzle8.max_fringe_len = 1
        Puzzle8.explored = set()
        res = mod_A(state[i],1,1)
        print("state:",i+1,", path length:",res.depth,", number of nodes generated:",Puzzle8.node_num,", maximum number of fringes:",Puzzle8.max_fringe_len)
        print(" ")
    print(" ")
    print("solving 10 states with depth = 1, 'manhattan distance' heuristic:")
    for i in range(10):
        Puzzle8.node_num = 1
        Puzzle8.max_fringe_len = 1
        Puzzle8.explored = set()
        res = mod_A(state[i],2,1)
        print("state:",i+1,", path length:",res.depth,", number of nodes generated:",Puzzle8.node_num,", maximum number of fringes:",Puzzle8.max_fringe_len)
        print(" ")
    print(" ")
    print("solving 10 states with depth = 5, 'mismatch count' heuristic:")
    for i in range(10):
        Puzzle8.node_num = 1
        Puzzle8.max_fringe_len = 1
        Puzzle8.explored = set()
        res = mod_A(state[i],1,5)
        print("state:",i+1,", path length:",res.depth,", number of nodes generated:",Puzzle8.node_num,", maximum number of fringes:",Puzzle8.max_fringe_len)
        print(" ")
    print(" ")
    print("solving 10 states with depth = 5, 'manhattan distance' heuristic:")
    for i in range(10):
        Puzzle8.node_num = 1
        Puzzle8.max_fringe_len = 1
        Puzzle8.explored = set()
        res = mod_A(state[i],2,5)
        print("state:",i+1,", path length:",res.depth,", number of nodes generated:",Puzzle8.node_num,", maximum number of fringes:",Puzzle8.max_fringe_len)
        print(" ")
    
    #Just here to show the path taken by the last state and to show the function.
    reproducePath(res)

In [50]:
driver()

solving 10 states with depth = 1, 'mismatch count' heuristic:
 
initial state:
5 3 8
4 e 2
6 7 1
state: 1 , path length: 22 , number of nodes generated: 10495 , maximum number of fringes: 3797
 
initial state:
e 5 1
3 8 2
4 7 6
state: 2 , path length: 18 , number of nodes generated: 1900 , maximum number of fringes: 725
 
initial state:
6 7 e
3 1 2
4 5 8
state: 3 , path length: 22 , number of nodes generated: 10901 , maximum number of fringes: 3940
 
initial state:
5 7 8
e 6 1
2 3 4
state: 4 , path length: 25 , number of nodes generated: 37671 , maximum number of fringes: 12220
 
initial state:
6 e 4
3 1 5
2 8 7
state: 5 , path length: 25 , number of nodes generated: 38878 , maximum number of fringes: 12324
 
initial state:
5 8 3
e 4 1
2 6 7
state: 6 , path length: 25 , number of nodes generated: 36130 , maximum number of fringes: 11604
 
initial state:
3 8 6
2 7 e
5 1 4
state: 7 , path length: 21 , number of nodes generated: 5367 , maximum number of fringes: 2015
 
initial state:
e 1 